In [5]:
import sys
sys.path.append("C:/Users/KL/NumericalMethods2")
import numpy as np
from NumericalMethodsFromSratch.Gaussian_Elim.Gaussian_Elimination import gauss

### Some helper functions

In [8]:
def read_input(name = 'input.txt'): 
    with open(name, "r") as f:
        n = int(f.readline())
        a = np.zeros((n, n), dtype=float)
        for i in range(n):
            a[i] = np.fromstring(f.readline(), sep=" ")
    
    return a

In [10]:
def _normalize_eigenvector_sign(v: np.ndarray) -> np.ndarray:
    v_complex = v.astype(complex, copy=False)
    max_abs_idx = np.argmax(np.abs(v_complex))
    phase = v_complex[max_abs_idx] / np.abs(v_complex[max_abs_idx])
    
    return v_complex / phase


def write_output(
    my_eigenvalue: complex, my_eigenvector: np.ndarray, 
    true_eigenvalue: complex, true_eigenvector: np.ndarray, 
    k: int,
    name: str = "output.txt"):
    
    try:
        # --- 1. Порівняння власних чисел (скаляри) ---
        abs_err_val = np.abs(my_eigenvalue - true_eigenvalue)
        rel_err_val = 0.0
        if not np.isclose(np.abs(true_eigenvalue), 0.0):
            rel_err_val = abs_err_val / np.abs(true_eigenvalue)
            
        # --- 2. Порівняння власних векторів ---
        
        norm_my_vec = _normalize_eigenvector_sign(my_eigenvector)
        norm_true_vec = _normalize_eigenvector_sign(true_eigenvector)
        
        abs_err_vec = np.linalg.norm(norm_my_vec - norm_true_vec)
        
        rel_err_vec = 0.0
        norm_true = np.linalg.norm(norm_true_vec)
        if not np.isclose(norm_true, 0.0):
            rel_err_vec = abs_err_vec / norm_true

        with open(name, "w", encoding="utf-8") as f:
            f.write(f"My Result:    {my_eigenvalue:.9e}\n")
            f.write(f"NumPy Result: {true_eigenvalue:.9e}\n")
            f.write(f"Absolute error: {abs_err_val:.6e}\n")
            f.write(f"Relative error: {rel_err_val:.6e}\n\n")
            
            f.write(f"My Result:    {np.array2string(norm_my_vec, precision=9, separator=', ', suppress_small=True)}\n")
            f.write(f"NumPy Result: {np.array2string(norm_true_vec, precision=9, separator=', ', suppress_small=True)}\n")
            f.write(f"Absolute error: {abs_err_vec:.6e}\n")
            f.write(f"Relative error: {rel_err_vec:.6e}\n")
            f.write(f"Iteration count: {k}\n")
            
    except Exception as e:
        with open(name, "w", encoding="utf-8") as f:
            f.write(f"Algorithm failed or an error occurred: {e}\n")
        return

### RQI Algorithm


In [13]:
def _find_x0_for_min_pair(A: np.ndarray, num_iters: int = 5) -> np.ndarray:
    n = A.shape[0]
    A_complex = A.astype(complex)

    x_k = np.random.rand(n) + 1j * np.random.rand(n)
    x_k = x_k / np.linalg.norm(x_k)

    for _ in range(num_iters):
        try:    
            y_k, _ = gauss(A_complex, x_k)
        except:
            return x_k
        
        norm_y_k = np.linalg.norm(y_k)
        x_k = y_k / norm_y_k

    return x_k

In [15]:
def rqi_iteration(A: np.ndarray, x: np.ndarray) -> complex:
    Ax = A @ x
    numerator = np.vdot(x, Ax)
    denominator = np.vdot(x, x)
    
    if np.isclose(denominator, 0.0):
        return 0.0
    return numerator / denominator

In [25]:
def rqi(A: np.ndarray, 
    x0: np.ndarray = None, tol: float = 1e-15) -> tuple[complex, np.ndarray]:
    
####----------------------------------####
    
    n = A.shape[0]
    if A.shape[0] != A.shape[1]:
        raise ValueError("Матриця A повинна бути квадратною.")
    A = A.astype(complex)
    
####----------------------------------####
    
    if x0 is None:
        x_k_minus_1 = np.random.rand(n) + 1j * np.random.rand(n)
    else:
        x_k_minus_1 = np.copy(x0).astype(complex)

    norm = np.linalg.norm(x_k_minus_1)
    if np.isclose(norm, 0.0):
        x_k_minus_1 = np.random.rand(n) + 1j * np.random.rand(n)
        norm = np.linalg.norm(x_k_minus_1)
        
    x_k_minus_1 = x_k_minus_1 / norm
    
####----------------------------------#### 
    
    rho_k_minus_1 = rqi_iteration(A, x_k_minus_1) #Відношення Релея
    E = np.identity(n, dtype=complex)
    
####----------------------------------#### 
    k = 0
    while True:
        k = k + 1
        #---#
        try:
            M = A - rho_k_minus_1 * E
            y_k, _ = gauss(M, x_k_minus_1)
            
        except np.linalg.LinAlgError:
            break # Матриця M стала сингулярною збіжність досягнута  
            
        #---#
        
        norm_y_k = np.linalg.norm(y_k)
        
        if np.isclose(norm_y_k, 0.0):
            break # Аварійний вихід
            
        #---#    
        
        x_k = y_k / norm_y_k
        rho_k = rqi_iteration(A, x_k)
        
        if np.abs(rho_k - rho_k_minus_1) < tol:
            rho_k_minus_1 = rho_k
            x_k_minus_1 = x_k
            break # Точність досягнуто
            
        rho_k_minus_1 = rho_k
        x_k_minus_1 = x_k
        
    return rho_k_minus_1, x_k_minus_1, k

In [27]:
def find_min_pair_with_rqi(A: np.ndarray, tol: float = 1e-15) -> tuple[complex, np.ndarray]:
    x0_primed = _find_x0_for_min_pair(A)
    
    min_eigenvalue, min_eigenvector, k = rqi(A, x0=x0_primed, tol=tol)
    
    return min_eigenvalue, min_eigenvector, k

In [29]:
A = read_input()

my_val, my_vec, k = find_min_pair_with_rqi(A)
true_vals, true_vecs = np.linalg.eig(A)
comparison_index = np.argmin(np.abs(true_vals - my_val))

write_output(my_val, my_vec, true_vals[comparison_index], true_vecs[:, comparison_index], k)

### Jacobi for eigenvalues/vectors Algorithm

In [218]:
def _normalize_real_vector(v: np.ndarray) -> np.ndarray:

    v_flat = np.asarray(v).flatten()
    
    if v_flat.size == 0:
        return v
        
    max_abs_idx = np.argmax(np.abs(v_flat))
    sign = np.sign(v_flat[max_abs_idx])
    
    if np.isclose(sign, 0.0):
        sign = 1.0
        
    return v / sign

def _normalize_eigenvector_matrix(V: np.ndarray) -> np.ndarray:
    V_normalized = V.copy()
    # Iterate over columns
    for i in range(V.shape[1]):
        V_normalized[:, i] = _normalize_real_vector(V_normalized[:, i])
        
    return V_normalized

def write_output_jacobi(my_eigenvalues: np.ndarray, 
                        my_eigenvectors: np.ndarray, 
                        true_eigenvalues: np.ndarray, 
                        true_eigenvectors: np.ndarray, 
                        name: str = "output_jacobi.txt"):

    try:
        my_eigenvalues = np.asarray(my_eigenvalues)
        true_eigenvalues = np.asarray(true_eigenvalues)
        
        idx_my = np.argsort(my_eigenvalues)
        idx_true = np.argsort(true_eigenvalues)
        
        my_evals_sorted = my_eigenvalues[idx_my]
        true_evals_sorted = true_eigenvalues[idx_true]
        

        my_evecs_sorted = my_eigenvectors[:, idx_my]
        true_evecs_sorted = true_eigenvectors[:, idx_true]


        abs_err_vals = np.linalg.norm(my_evals_sorted - true_evals_sorted)
        rel_err_vals = 0.0
        norm_true_vals = np.linalg.norm(true_evals_sorted)
        if not np.isclose(norm_true_vals, 0.0):
            rel_err_vals = abs_err_vals / norm_true_vals
            

        norm_my_vecs = _normalize_eigenvector_matrix(my_evecs_sorted)
        norm_true_vecs = _normalize_eigenvector_matrix(true_evecs_sorted)
        

        abs_err_vecs = np.linalg.norm(norm_my_vecs - norm_true_vecs)
        
        rel_err_vecs = 0.0
        norm_true_vecs_val = np.linalg.norm(norm_true_vecs)
        if not np.isclose(norm_true_vecs_val, 0.0):
            rel_err_vecs = abs_err_vecs / norm_true_vecs_val

        
        with open(name, "w", encoding="utf-8") as f:
            f.write("--- Eigenvalues Comparison ---\n")
            f.write("My Result:    " + np.array2string(my_evals_sorted, precision=9, separator=', ', suppress_small=True) + "\n")
            f.write("NumPy Result: " + np.array2string(true_evals_sorted, precision=9, separator=', ', suppress_small=True) + "\n")
            f.write(f"Absolute L2 error: {abs_err_vals:.6e}\n")
            f.write(f"Relative L2 error: {rel_err_vals:.6e}\n\n")
            
            f.write("--- Eigenvectors Comparison ---\n")
            f.write("My Result (first 5 rows/cols):\n" + np.array2string(norm_my_vecs[:5, :5], precision=5, suppress_small=True) + "\n\n")
            f.write("NumPy Result (first 5 rows/cols):\n" + np.array2string(norm_true_vecs[:5, :5], precision=5, suppress_small=True) + "\n\n")
            f.write(f"Absolute Matrix Norm Diff: {abs_err_vecs:.6e}\n")
            f.write(f"Relative Matrix Norm Diff: {rel_err_vecs:.6e}\n")
            
    except Exception as e:
        import traceback
        with open(name, "w", encoding="utf-8") as f:
            f.write(f"Algorithm failed or an error occurred: {e}\n")
            f.write(traceback.format_exc())
        return

In [212]:
def jacobi(A: np.ndarray, tol: float = 1e-15) -> tuple[np.ndarray, np.ndarray]:
    
    A = np.asarray(A, dtype=float)
    if not np.allclose(A, A.T):
        raise ValueError("Input matrix A must be symmetric.")
        
    n = A.shape[0]
    B = A.copy()
    V = np.identity(n)
    
    iteration = 0
    
    while True:
        iteration += 1
        
        off_diag_mask = np.ones((n, n), dtype=bool)
        np.fill_diagonal(off_diag_mask, False)
        
        B_upper = np.triu(np.abs(B), k=1)
        i, j = np.unravel_index(np.argmax(B_upper), (n, n))
        max_val = B_upper[i, j]

        if max_val < tol:
            break 

        ####----------------------------------####
        
        b_ii = B[i, i]
        b_jj = B[j, j]
        b_ij = B[i, j]
        
        q = b_jj - b_ii 
        
        if np.abs(b_ij) < tol:
             continue

        if np.isclose(q, 0.0):
            t = 1.0 if b_ij > 0 else -1.0
        else:
            tau = q / (2 * b_ij)
            t = np.sign(tau) / (np.abs(tau) + np.sqrt(1 + tau**2))
            
        c = 1.0 / np.sqrt(1 + t**2)
        s = c * t
        
        ####----------------------------------####
        
        
        B_ik = B[i, :].copy()
        B_jk = B[j, :].copy()
        
        B[i, :] = c * B_ik - s * B_jk
        B[j, :] = s * B_ik + c * B_jk
        
        B[:, i] = B[i, :]
        B[:, j] = B[j, :]

        B[i, i] = c**2 * b_ii + s**2 * b_jj - 2 * c * s * b_ij
        B[j, j] = s**2 * b_ii + c**2 * b_jj + 2 * c * s * b_ij
        B[i, j] = 0.0
        B[j, i] = 0.0
        
        ####----------------------------------####
        
        V_ki = V[:, i].copy()
        V_kj = V[:, j].copy()
        
        V[:, i] = c * V_ki - s * V_kj
        V[:, j] = s * V_ki + c * V_kj
        
    eigenvalues = np.diag(B)
    eigenvectors = V
    
    return eigenvalues, eigenvectors

In [214]:
A = read_input("input_jacobi.txt")

my_val, my_vec = jacobi(A)
true_vals, true_vecs = np.linalg.eigh(A)

write_output_jacobi(my_val, my_vec, true_vals, true_vecs)